### Random Forest Classifier

In [1]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
# Import sklearn dependencies
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# Load the avocado_mock_df.csv dataset.
avocado_mock_df = pd.read_csv(Path("./MOCK_datasets/MockDataset_Avocado_ML.csv"))
avocado_mock_df.head(10)

,REGION (city),LATITUDE,LONGITUDE,DATE (Month + Year),4046 Units sold,4225 Units,4770 Units,Bags sold,CLIMATE_CA_pcp,CLIMATE_CA_tmin,...,PERU PRODUCTION,COLUMBIA PRODUCTION,IMPORTS BY VALUE ($1000),IMPORTS BY VOLUME (1000 LBS),IMPORTS UNIT VALUE ($ PER POUND),EXPORTS BY VALUE ($1000),EXPORTS BY VOLUME (1000 LBS),EXPORTS UNIT VALUE ($ PER POUND),MEDIAN INCOME,PRICE
0,Los Angeles,34.0522,118.2437,1/1/2017,1223299.39,829896.69,56808.74,1441332.35,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,64251,0.84
1,San Diego,32.7157,117.1611,1/1/2017,149666.20,206181.12,15014.45,189448.58,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,74855,0.93
2,San Francisco,37.7749,122.4194,1/1/2017,260100.73,717625.67,2490.66,66958.27,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,104552,0.95
3,Sacramento,38.5816,121.4944,1/1/2017,144876.31,339664.16,660.97,41564.20,6.48,36.6,...,0,0,207387693,214329.7208,0.967611,1660842,994.194413,1.670540,63902,0.98
4,Los Angeles,34.0522,118.2437,2/1/2017,5470227.08,1741607.02,937331.61,2701609.82,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,64251,0.53
5,San Diego,32.7157,117.1611,2/1/2017,260418.91,290223.93,20313.95,321784.08,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,74855,0.63
6,San Francisco,37.7749,122.4194,2/1/2017,560686.25,900172.65,21331.13,75785.02,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,104552,0.84
7,Sacramento,38.5816,121.4944,2/1/2017,321474.22,418158.93,11962.44,57213.28,3.65,37.5,...,0,0,182421005,157957.1830,1.154876,2119913,1400.845874,1.513309,63902,0.98
8,Los Angeles,34.0522,118.2437,3/1/2017,2768930.88,897311.08,518369.51,1269943.15,2.19,37.2,...,0,0,224948272,171411.0822,1.312332,4549077,2623.655244,1.733870,64251,0.99
9,San Diego,32.7157,117.1611,3/1/2017,163424.69,176304.67,19186.14,158785.76,2.19,37.2,...,0,0,224948272,171411.0822,1.312332,4549077,2623.655244,1.733870,74855,1.04


In [5]:
# Create our features
dfd = df.drop(columns="PRICE")
X = pd.get_dummies(dfd, columns=["home_ownership","verification_status", "issue_d","pymnt_plan", "initial_list_status", "next_pymnt_d","application_type","hardship_flag", "debt_settlement_flag"])


# Create our target
y = df["PRICE"]

NameError: name 'df' is not defined

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_test)

Counter({'low_risk': 17104, 'high_risk': 101})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)  

BalancedRandomForestClassifier(random_state=1)

In [11]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)  

0.7885466545953005

In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   71,    30],
       [ 2153, 14951]], dtype=int64)

In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.62     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.62     17205



In [14]:
# List the features sorted in descending order by feature importance
feature_importance = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
for i in feature_importance:
    print('{} : ({})'.format(i[1], i[0]))

total_rec_prncp : (0.07876809003486353)
total_pymnt : (0.05883806887524815)
total_pymnt_inv : (0.05625613759225244)
total_rec_int : (0.05355513093134745)
last_pymnt_amnt : (0.0500331813446525)
int_rate : (0.02966959508700077)
issue_d_Jan-2019 : (0.021129125328012987)
installment : (0.01980242888931366)
dti : (0.01747062730041245)
out_prncp_inv : (0.016858293184471483)
total_bal_ex_mort : (0.01641297102011915)
mths_since_recent_inq : (0.015220714904737209)
issue_d_Mar-2019 : (0.015115240704562424)
revol_bal : (0.014926655663448373)
max_bal_bc : (0.014899352873994727)
out_prncp : (0.014881069023035237)
mo_sin_old_rev_tl_op : (0.014859446582326507)
annual_inc : (0.014832564501144122)
bc_util : (0.014613819728800227)
tot_hi_cred_lim : (0.014487685026878092)
mo_sin_old_il_acct : (0.013921085423763812)
mths_since_rcnt_il : (0.013534131593418711)
total_rev_hi_lim : (0.013364759441576994)
avg_cur_bal : (0.01332289882475225)
bc_open_to_buy : (0.013265926832893358)
il_util : (0.01311545089813887

### Easy Ensemble AdaBoost Classifier

In [15]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train) 


EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [16]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)  

0.9316600714093861

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = eec.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [  983, 16121]], dtype=int64)

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

